This is a depreciated notebook. Anything good in it has been moved over to vis_methods, what's left is me calling a couple vis methods functions and some really bad functions that should not see the light of day

Import packages

In [1]:
import env_methods as em
import vis_methods as vis
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import numpy as np
import math
import time
import datetime as dt

Basic parameters

In [2]:
#Define bounding box in North America
lat_start = 0
lat_end = 90
lon_start = -179
lon_end =  -30

#Data paths
main_path = r'C:\Users\bpara\Geo data'

temps_path = r'\temp'
geomean_path = r'\geopotential mean'
rain_path = r'\surface precip'
snow_path = r'\snow depth'
moisture_path = r'\soil moisture'
shum_path = r'\specific humidity'

#Time stuff
month_to_show = 4 #What number of month to show, 0 = Dec 1843, 1 = Jan 1844, 2 = Feb 1844...
months = ["December", "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November"]

Import data

In [3]:
temp_data = em.get_data(main_path+temps_path, False, [12, 1843, 12, 1844])
mean_data = em.get_data(main_path+geomean_path, False, [12, 1843, 12, 1844])
wind_data = em.get_data(main_path+r'\wind', False, [12, 1843, 12, 1844])

acquring data from air.2m.mon.mean.nc
acquring data from hgt.mon.mean.nc
acquring data from uwnd.mon.mean.nc
acquring data from vwnd.mon.mean.nc


Draw the mapping

In [4]:
vis.plot_data(shum_data["lon"], shum_data["lat"], 
              shum_data["shum"][em.find_closest_val(dt.datetime(1994, 1,1), rain_data["time"])][1], 
          "Geopotential Height (m)", "Geopotential Height Mean at 850mb in January 1994", lon_start, lon_end, lat_start, lat_end)
vis.plot_data(temp_data["lon"], temp_data["lat"], temp_data["air"][month_to_show], "Temperature, (K)", 
          "Average Surface Temp in April 1844", lon_start, lon_end, lat_start, lat_end)

NameError: name 'shum_data' is not defined

And now for vector wind

In [ ]:
vis.plot_vector_data(wind_data["lon"], wind_data["lat"], wind_data["nc_vars"]["uwnd"][month_to_show][0], 
                 wind_data["nc_vars"]["vwnd"][month_to_show][0], "Vector wind at 1000mb in April 1844", "Wind Speed (m/s)",
                 lon_start, lon_end, lat_start, lat_end)

Plain vector wind can get a little messy with really long arrows, this one normalizes the length of all the arrows and only uses the color for magnitude

In [ ]:
vis.plot_nvector_data(wind_data["lon"], wind_data["lat"], wind_data["nc_vars"]["uwnd"][month_to_show][1], 
                 wind_data["nc_vars"]["vwnd"][month_to_show][1], "Vector wind at 850mb in April 1844", "Wind Speed (m/s)",
                 lon_start, lon_end, lat_start, lat_end)

In [ ]:
wind_data

In [ ]:
#this is a terrible function I """wrote"""" at 1:00 AM for 1 specific purpose. It is a bastardized version of the 
#generate 6 month plot for vector fields with like half of the parameters not doing anything. At some point, I might
#update it to something worthwhile, but until then, look upon it and weep weary traveler. 

def generate_six_month_plot_vector(fname, scalebar, title, data, datakey, start_year, end_year, focus_year, has_levels, level=0, 
                           lon_min = 0, lon_max = 359, lat_min = -90, lat_max = 90, 
                            has_scale_bounds = False, scale_min = -3, scale_max = 3):
    
    lat1 = em.find_closest_val(lat_min, data["lat"])
    lat2 = em.find_closest_val(lat_max, data["lat"])
    
    mesh = None
    
    months2 = [12, 1, 2, 3, 4, 5]
    years = [1843, 1844, 1844, 1844, 1844, 1844]
    
    fig, axs = plt.subplots(2,3, figsize=(30, 12), subplot_kw={'projection':ccrs.PlateCarree(central_longitude = 0)})
    for x in range(len(axs)):
        for y in range(len(axs[x])):
            axs[x][y].coastlines()
            axs[x][y].add_feature(cartopy.feature.LAKES, alpha=1)
            axs[x][y].add_feature(cartopy.feature.RIVERS, alpha = 1)
            axs[x][y].set_extent((lon_min, lon_max, lat_min, lat_max))
            axs[x][y].set_xticks(data["lon"][math.ceil(lon_min / 10) * 10:lon_max][::20], crs = ccrs.PlateCarree())
            axs[x][y].set_yticks(data["lat"][math.ceil(lat1 / 10) * 10:lat2][::20], crs = ccrs.PlateCarree())
            axs[x][y].set_title(months[3*x+y])
            time_data = None
            time_data = [months2[3*x+y], years[3*x+y]]
             
            
            cguide = np.hypot(data["nc_vars"]["uwnd"][3*x+y, level, lat1:lat2, lon_min:lon_max][::3,::3], 
                     data["nc_vars"]["vwnd"][3*x+y, level, lat1:lat2, lon_min:lon_max][::3,::3])
            
            mesh = axs[x][y].quiver(data["lon"][lon_min:lon_max][::3], 
                              data["lat"][lat1:lat2][::3], 
                              data["nc_vars"]["uwnd"][3*x+y, level, lat1:lat2, lon_min:lon_max][::3,::3], 
                                    data["nc_vars"]["vwnd"][3*x+y, level, lat1:lat2, lon_min:lon_max][::3,::3], cguide,
                                    cmap="GnBu", scale = 300)
            
            
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    bar = fig.colorbar(mesh, cax=cbar_ax)
    bar.set_label(scalebar)
    
    fig.suptitle(title)
    
    plt.savefig(fname)
            
generate_six_month_plot_vector("wind 1000mb.png", "Wind Speed (m/s)", "Vector wind at 1000mb in 1843-1844",
                        wind_data, "soilm", 1836, 1980, 1844, has_levels=False, level = 0, lon_min = lon_start, lon_max = lon_end, 
                        lat_min = lat_start, lat_max = lat_end, has_scale_bounds = True, scale_min = 3, scale_max = 3)